### Importing the Libraries

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
tf.__version__ 

'2.13.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [3]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

#### Generating images for the Test set

In [4]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [5]:
training_set = train_datagen.flow_from_directory(r'C:/Users/ABHISHEK PATIL/Mega Project/Sign-Language-To-Text-Conversion-main\dataSet/trainingData',
                                                 target_size=(128, 128),
                                                 batch_size=5,
                                                 color_mode='grayscale',
                                                 class_mode='categorical')


Found 12845 images belonging to 27 classes.


In [6]:
test_set = test_datagen.flow_from_directory(r'C:/Users/ABHISHEK PATIL/Mega Project/Sign-Language-To-Text-Conversion-main/dataSet/testingData',
                                            target_size=(128, 128),
                                            batch_size=5,
                                            color_mode='grayscale',
                                            class_mode='categorical')


Found 4268 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [7]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [8]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [9]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [10]:
classifier.add(tf.keras.layers.Conv2D(filters=64, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [11]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [12]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.50))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.50))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [13]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [14]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 64, 64, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 32, 32, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 65536)             0         
                                                                 
 dense (Dense)               (None, 128)               8

In [15]:
classifier.fit(training_set,
                  epochs = 40,
                  validation_data = test_set)

Epoch 1/40
2569/2569 [==============================] - 515s 200ms/step - loss: 2.5997 - accuracy: 0.2210 - val_loss: 0.7103 - val_accuracy: 0.8020
Epoch 2/40
2569/2569 [==============================] - 510s 199ms/step - loss: 1.1019 - accuracy: 0.6281 - val_loss: 0.2077 - val_accuracy: 0.9222
Epoch 3/40
2569/2569 [==============================] - 532s 207ms/step - loss: 0.7333 - accuracy: 0.7504 - val_loss: 0.1419 - val_accuracy: 0.9646
Epoch 4/40
2569/2569 [==============================] - 489s 190ms/step - loss: 0.5807 - accuracy: 0.8030 - val_loss: 0.0713 - val_accuracy: 0.9773
Epoch 5/40
2569/2569 [==============================] - 1142s 445ms/step - loss: 0.4935 - accuracy: 0.8360 - val_loss: 0.0483 - val_accuracy: 0.9864
Epoch 6/40
2569/2569 [==============================] - 511s 199ms/step - loss: 0.4325 - accuracy: 0.8588 - val_loss: 0.0165 - val_accuracy: 0.9977
Epoch 7/40
2569/2569 [==============================] - 2322s 904ms/step - loss: 0.3831 - accuracy: 0.8762 - va

#### Saving the Model

In [16]:
# model_json = classifier.to_json()
# with open("model_new.json", "w") as json_file:
#     json_file.write(model_json)
# print('Model Saved')
# classifier.save_weights('model_new.h5')
# print('Weights saved')

In [17]:
model_json = classifier.to_json()

# Specify the destination path for the JSON file (model architecture)
destination_json_path = "model_new.json"
with open(destination_json_path, "w") as json_file:
    json_file.write(model_json)
print('Model architecture saved at:', destination_json_path)

# Specify the destination path for the HDF5 file (model weights)
destination_weights_path = "model_new.h5"
print('Model weights saved at:', destination_weights_path)


Model architecture saved at: model_new.json
Model weights saved at: model_new.h5
